In [291]:
import pandas as pd
import seaborn as sns
import openpyxl
import math
import re
import numpy as np
import heapq

In [292]:
zipdata = pd.read_csv("ExternalDataByZipcode.csv")

# convert data into usuable types
zipdata['Median Household Income'] = zipdata['Median Household Income'].str.replace('\W', '', regex=True).astype(float)
zipdata['Population'] = zipdata['Population'].str.replace('\W', '', regex=True).astype(float)
zipdata['White Percentage'] = (zipdata['White Percentage'].str.replace('\W', '', regex=True).astype(float)) / 100

# print(zipdata.var())
# print(zipdata.dtypes)
zipdata.head()

focusdf = zipdata[['Zip Codesort column', 'Median Household Income', 'Median Age', 'Population', "White Percentage"]]
# normed_zipdata = (focusdf-focusdf.mean())/focusdf.std()
# normed_zipdata
normed_zipdata = (focusdf-focusdf.min())/(focusdf.max()-focusdf.min())
normed_zipdata

,Zip Codesort column,Median Household Income,Median Age,Population,White Percentage
0,0.000000,0.291734,0.636015,0.164170,0.586141
1,0.001678,0.285460,0.593870,0.116437,0.593670
2,0.003356,0.118556,0.586207,0.165117,0.313298
3,0.005034,0.812803,0.659004,0.309294,0.864599
4,0.006711,0.305435,0.793103,0.272116,0.882593
...,...,...,...,...,...
128,0.939597,0.167568,0.593870,0.120597,0.783946
129,0.954698,0.289195,0.624521,0.479409,0.889357
130,0.979866,0.410545,0.793103,0.315425,0.940531
131,0.981544,0.094597,0.402299,0.178032,0.736983


In [430]:

# for each zip code find standardized euclidean norm (2-norm) of varialbes
# vars = [income, age, population, nonwhite]

# initialize list of norms
norm = []

# select which zipcode to test
zipcode = 77093
zipindex = zipdata[zipdata['Zip Codesort column'] == zipcode].index.values
targeted = [zipindex[0]]

for precinct in targeted:
    # retrieves values of variables for zipcodes of targeted precincts
    tar_income = zipdata.iloc[precinct]['Median Household Income']
    tar_age = zipdata.iloc[precinct]['Median Age']
    tar_pop = zipdata.iloc[precinct]['Population']
    tar_div = 100 - zipdata.iloc[precinct]['White Percentage']

    for i in range(zipdata.shape[0]):
        # takes out testing precinct with itself
        if precinct != i:
            # retrieves values of variables for other zipcodes
            income = zipdata.iloc[i]['Median Household Income']
            age = zipdata.iloc[i]['Median Age']
            pop = zipdata.iloc[i]['Population']
            div = 100 - zipdata.iloc[i]['White Percentage']
            # standardizes the difference between the targeted and other zipcodes
            # i.e. income and age are varied across vastly different scales
            incomediff = (tar_income - income) / (1.067543e+09)
            agediff = (tar_age - age) / (1.693905e+01)
            popdiff = (tar_pop - pop) / (1.558452e+08)
            divdiff = (tar_div - div) / (3.090548e+02)
            # sums the standardized differences
            sum = ((incomediff)**2) + ((agediff)**2) + ((popdiff)**2) + ((divdiff)**2)
        # takes the square root of the sum according to formula
        norm.append(math.sqrt(sum))

# print(norm)
print('standardized euclidean norm:', min(norm))

smallest = heapq.nsmallest(10, norm)
print(smallest)

topzips = {}
ziptop = []

for zip in smallest:
    index = norm.index(zip)
    code = zipdata.loc[index].at['Zip Codesort column']
    if zipcode in topzips:
        topzips[zipcode].append(code)
    else:
        topzips[zipcode] = [code]
    ziptop.append(code)
        
print(topzips)

indexnorm = np.argmin(norm)
print('targeted zipcode:', zipdata.loc[[targeted[0]]])
print('closest other zipcode:', zipdata.loc[[indexnorm]])


standardized euclidean norm: 0.014579789242298453
[0.014579789242298453, 0.03475239910668372, 0.035436243828508567, 0.03872870220911158, 0.03918656364580447, 0.05108487790743446, 0.05920382170297626, 0.06398957083279007, 0.07276829631771195, 0.07284657357175957]
{77093: [77038, 77039, 77081, 77076, 77037, 77073, 77075, 77086, 77530, 77547]}
targeted zipcode:     Zip Codesort column  Median Household Income  Median Age  Population  \
90                77093                  36152.0        29.9     25052.0   

    White Percentage Black/AfricanAmerican Percentage  \
90             50.21                           10.82%   

   Pacific Islander Percentage Asian Percentage Other Race Population  \
90                       0.05%            0.36%                33.78%   

   2+ Races Percentage     sum  
90               3.71%  98.93%  
closest other zipcode:     Zip Codesort column  Median Household Income  Median Age  Population  \
36                77038                  45709.0        30.

In [340]:
vote_sheet = pd.read_csv('cleaned_nov_votes.csv')

# dict maps zip to a set of precincts in that zip
zip_to_prec_dict = {}

# the reverse lol - doct that maps precincts to the zipcodes that they are in
prec_to_zip_dict = {}

for idx, row in vote_sheet.iterrows():

    zipcode = row.loc["ZIP"]
    if zipcode not in zip_to_prec_dict.keys():
        zip_to_prec_dict[zipcode] = {row.loc["Voting Precinct"]}
    else:
        zip_to_prec_dict[zipcode].add(row.loc["Voting Precinct"])

    precinct = row.loc["Voting Precinct"]

    if precinct not in prec_to_zip_dict.keys():
        prec_to_zip_dict[precinct] = {row.loc["ZIP"]}
    else:
        prec_to_zip_dict[precinct].add(row.loc["ZIP"])

In [422]:
key_list = list(prec_to_zip_dict.keys())
val_list = list(prec_to_zip_dict.values())

#print(val_list)

precs = {}
print(ziptop)
for val in ziptop:
    if {val} in val_list:
        ogindex = val_list.index({zipcode})
        index = val_list.index({val})
        
    #ogindex = zipcode
        ogindex = zipcode
        if ogindex in precs:
            precs[ogindex].append(index)
            print(val, zipcode)
        else:
            print('ah', val, zipcode, index)
            precs[ogindex] = [index]

print(precs)

# for zip in smallest:
#     index = norm.index(zip)
#     code = zipdata.loc[index].at['Zip Codesort column']
#     if zipcode in topzips:
#         topzips[zipcode].append(code)
#     else:
#         topzips[zipcode] = [code]
# print(val_list)

# index = val_list.index({77037})
# print(key_list[index])


# find zip from prec
# x = prec_to_zip_dict[46]
# print(x)

# key_list2 = list(zip_to_prec_dict.keys())
# val_list2 = list(zip_to_prec_dict.values())

# index2 = val_list2.index(10)
# print(key_list2[index2])

[77022, 77080, 77520, 77338, 77053, 77045, 77072, 77085, 77061, 77011]
ah 77022 77074 282
77080 77074
77520 77074
77338 77074
77045 77074
77072 77074
77061 77074
77011 77074
{77074: [282, 82, 186, 181, 375, 660, 52, 20]}


In [431]:
import plotly.graph_objects as go

categories = ['median household income','median age','total population',
              'underrepresented voter percentage']

print(zipindex)
og_income = normed_zipdata.loc[zipindex[0]].at['Median Household Income']
og_age = normed_zipdata.loc[zipindex[0]].at['Median Age']
og_pop = normed_zipdata.loc[zipindex[0]].at['Population']
og_div = 1 - normed_zipdata.loc[zipindex[0]].at['White Percentage']

fig = go.Figure()
fig.add_trace(go.Scatterpolar(
      r=[og_income, og_age, og_pop, og_div],
      theta=categories,
      fill='toself',
      name='77093'
))

sim_income = normed_zipdata.loc[indexnorm].at['Median Household Income']
sim_age = normed_zipdata.loc[indexnorm].at['Median Age']
sim_pop = normed_zipdata.loc[indexnorm].at['Population']
sim_div = 1 - normed_zipdata.loc[indexnorm].at['White Percentage']

fig.add_trace(go.Scatterpolar(
      r=[sim_income, sim_age, sim_pop, sim_div],
      theta=categories,
      fill='toself',
      name='77038'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True
)



[90]


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'fill': 'toself',
              'name': '77093',
              'r': [0.01689332766680833, 0.35632183908045967, 0.3833998230390711,
                    0.45380296069423176],
              'theta': [median household income, median age, total population,
                        underrepresented voter percentage],
              'type': 'scatterpolar'},
             {'fill': 'toself',
              'name': '77038',
              'r': [0.06766362090947727, 0.3601532567049808, 0.23228450350041135,
                    0.5063808065339458],
              'theta': [median household income, median age, total population,
                        underrepresented voter percentage],
              'type': 'scatterpolar'}],
    'layout': {'polar': {'radialaxis': {'range': [0, 1], 'visible': True}}, 'showlegend': True, 'template': '...'}
})

In [179]:
import plotly.graph_objects as go

categories = ['median household income','median age','total population',
              'underrepresented voter percentage']

# fig = go.Figure()

# zipdata.loc[zipdata.n == "closezip", ['a','b']].values.flatten().tolist()
# r = zipdata.loc[0, 2:6].values.tolist()

# r = zipdata.values.tolist()[0]
# [1, 6]

#r = zipdata.loc[indexnorm, 2:6].values.flatten().tolist()

#print(r)

# scat_income = zipdata.iloc[indexnorm]['Median Household Income'] / (1.067543e+09)
# scat_age = zipdata.iloc[indexnorm]['Median Age'] / (1.693905e+01)
# scat_pop = zipdata.iloc[indexnorm]['Population'] / (1.558452e+08)
# scat_div = 100 - zipdata.iloc[indexnorm]['White Percentage'] / (3.090548e+02)


# fig.add_trace(go.Scatterpolar(
#       r=[scat_income, scat_age, scat_pop, scat_div],
#       theta=categories,
#       fill='toself',
#       name='77026'
# ))

,Zip Codesort column,Median Household Income,Median Age,Population,White Percentage
0,0.000000,0.291734,0.636015,0.164170,0.586141
1,0.001678,0.285460,0.593870,0.116437,0.593670
2,0.003356,0.118556,0.586207,0.165117,0.313298
3,0.005034,0.812803,0.659004,0.309294,0.864599
4,0.006711,0.305435,0.793103,0.272116,0.882593
...,...,...,...,...,...
128,0.939597,0.167568,0.593870,0.120597,0.783946
129,0.954698,0.289195,0.624521,0.479409,0.889357
130,0.979866,0.410545,0.793103,0.315425,0.940531
131,0.981544,0.094597,0.402299,0.178032,0.736983
